### **1. Import Dependencies**

In [292]:
import pandas as pd
import numpy as np

import gc
import os
import requests
import json
import re

import concurrent.futures
import asyncio
import httpx
import multiprocessing

from urllib.parse import urlencode

from typing import List, Dict, Optional, Union
from glob import glob

import joblib

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.metrics import precision_score, recall_score

from sentence_transformers import SentenceTransformer
import fasttext
from gensim.models import KeyedVectors 

import tensorflow as tf
import tensorflow.keras as keras

from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, LSTM, Bidirectional, Input, Lambda, Dot

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\msi.pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\msi.pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### **2. Settings**

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

### **2. Load Data**

In [ ]:
def fetch_data(url: str, params=None):
  try:
    response = requests.get(url, params=params)
    if (response.status_code == 200):
      return response.json()
    
    print(f"Error response with status code: {response.status_code}")
  except Exception as error:
    print(f'Failed to fetch data: {error}')

def urls_builder(base_url: str, n_fetch: int, limit: int, products: List[str], **kwargs):
  urls = []
  for product in products:
    for i in range(n_fetch):
      
      param = {
        'offset': i * limit,
        'limit': limit,
        'product': product,
        **kwargs,
      }
      
      full_url = base_url + '?' + urlencode(param)
      urls.append(full_url)
    
  return urls

def save_json(data, path: str):
  with open(path, 'w') as json_file:
    json.dump(data, json_file)

def load_json(path: str):
  with open(path, 'r') as json_file:
    loaded_data = json.load(json_file)
  return loaded_data

In [ ]:
%%script skipped # Skipped due to resource optimization

selected_fields = ['id', 'duplicates', 'summary', 'description', 'status', 'resolution', 'platform', 'product', 'type', 'priority', 'severity', 'component']
products = ['Firefox']

base_params = {
  'include_fields': ', '.join(selected_fields),
}
saved_data_path = os.path.join('data', 'raw_data', 'firefox_raw_data.json')

base_url = 'https://bugzilla.mozilla.org/rest/bug'
n_fetch = 50
limit = 5000

if os.path.exists(saved_data_path):
  response_data = load_json(saved_data_path)
else:
  urls = urls_builder(base_url, n_fetch, limit, products, **base_params)
  response_data = []

  max_workers = 50
  with concurrent.futures.ThreadPoolExecutor(max_workers) as executor:
    response_data = list(executor.map(fetch_data, urls))
    
  response_data = [item['bugs'] for item in response_data]
  response_data = [item for sublist in response_data for item in sublist]
  
  save_json(response_data, saved_data_path)

In [ ]:
raw_data_path = os.path.join('data', 'cache', 'raw_data.parquet')
df = pd.DataFrame()

if (os.path.exists(raw_data_path)):
  df = pd.read_parquet(raw_data_path)
else:
  data_paths = glob(os.path.join('data', 'raw_data', 'firefox_raw_data.json'))

  for path in data_paths:
    data = load_json(path)
    data = pd.DataFrame(data)
    df = pd.concat([df, data])

  df = df.set_index('id')
  df.to_parquet(raw_data_path)

df.head()

In [ ]:
df = df.drop(['status', 'priority', 'resolution', 'severity', 'component', 'product'], axis=1)

In [ ]:
df.info(memory_usage='deep')

### **3. EDA**

### **3.1. Missing Values**

In [ ]:
df.isna().sum()

### **3.2. Product**

In [ ]:
%%script skipped # Skipped due to resource optimization

df['product'].value_counts(ascending=False)

### **3.3. Platform**

In [ ]:
df['platform'].value_counts(ascending=False)

### **3.4. Type**

In [ ]:
df['type'].value_counts(ascending=False)

### **3.5. Duplicates**

In [ ]:
duplicates = df['duplicates'].apply(lambda x: len(x)).sort_values(ascending=False)
duplicates

In [ ]:
duplicated_bugs = duplicates[duplicates > 0]
duplicated_bugs.count()

### **4. Feature Engineering**

### **4.1. Missing Values**

In [ ]:
df.loc[df['type'] == '--', 'type'] = 'no type'

### **4.2. Duplicates**

In [ ]:
duplicated = df.loc[duplicated_bugs.index, 'duplicates']
duplicated

In [70]:
data_path = os.path.join('data', 'cache', 'duplicate_to_data.parquet')

if (os.path.exists(data_path)):
  df = pd.read_parquet(data_path)
else:
  df['duplicates_to'] = -1

  for idx, dups in zip(duplicated.index, duplicated):
    for item in dups:
      df.loc[df.index == item, 'duplicates_to'] = idx
      
  df.to_parquet(data_path)

In [71]:
df = df.drop(columns=['duplicates'])

### **4.3. Clean Sentences**

In [72]:
def remove_special_chars(text: str):
  text = text.lower()
  text = re.sub(r'\n|\t|\r|\0', ' ', text)
  text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
  text = re.sub(r'\s{2,}', ' ', text)
  text = re.sub(r'\s$', '', text)
  text = re.sub(r'\s[b-z]\s', ' ', text)
  text = re.sub(r'\s[b-z]\s', ' ', text)
  
  return text

def remove_stopwords(text: str):
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text)
  filtered_words = [word for word in words if word not in stop_words]
  
  return ' '.join(filtered_words)

def clean_text(text: str):
  text = remove_special_chars(text)
  text = remove_stopwords(text)
  
  return text

df['platform'] = df['platform'].apply(clean_text)
df['summary'] = df['summary'].apply(clean_text)
df['type'] = df['type'].apply(clean_text)
df['description'] = df['description'].apply(clean_text)

# df['product'] = df['product'].apply(clean_text)

### **4.4. Combined Text**

In [73]:
df['text'] = df['type'] + ' ' + df['platform'] + ' ' + df['summary'] + ' ' + df['description']

In [74]:
df = df[['text', 'duplicates_to']]

df.head()

,text,duplicates_to
id,,
10954,enhancement dialup properties needs exposed p...,-1
14871,defect find find whole word please add match ...,-1
19118,enhancement plug manager ui choosing mimetype...,-1
21482,enhancement improvement save file dialog fold...,-1
23207,enhancement unspecified options save location ...,-1


### **4.5. Sentence Embedding**

In [135]:
%%script skipped # Skipped

sent_embd_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [136]:
%%script skipped # Skipped

embd_data_path = os.path.join('data', 'cache', 'sent_trans_embd_data.parquet')

if (os.path.exists(embd_data_path)):
  df = pd.read_parquet(embd_data_path)
else:
  df['text_embedded'] = df['text'].apply(sent_embd_model.encode)
  df.to_parquet(embd_data_path)

df['text_embedded'] = df['text_embedded'].apply(np.array)
gc.collect()

16

In [167]:
# %%script skipped # Skipped

model_path = os.path.join('models', 'wiki-news-300d-1M-subword.vec')
sent_embd_model = KeyedVectors.load_word2vec_format(model_path, binary=False)

In [168]:
# %%script skipped # Skipped

def sentence_embedding(sentence, model):
    words = word_tokenize(sentence)
    words_in_vocab = [word for word in words if word in model]

    if not words_in_vocab:
        return None

    embedding = np.mean([model[word] for word in words_in_vocab], axis=0)
    embedding = normalize(embedding.reshape(1, -1), norm='l2').reshape(-1)
    
    return embedding

In [252]:
# %%script skipped # Skipped

embd_data_path = os.path.join('data', 'cache', 'fasttext_embd_data.parquet')

if (os.path.exists(embd_data_path)):
  df = pd.read_parquet(embd_data_path)
else:
  df['text_embedded'] = df['text'].apply(sentence_embedding, model=sent_embd_model)
  df.to_parquet(embd_data_path)

df['text_embedded'] = df['text_embedded'].apply(np.array)
gc.collect()

0

In [253]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 210693 entries, 10954 to 1876223
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   text           210693 non-null  object
 1   duplicates_to  210693 non-null  int64 
 2   text_embedded  210693 non-null  object
dtypes: int64(1), object(2)
memory usage: 450.2 MB


### **4.6. Sentence Pairs**

In [254]:
df = df.reset_index()

In [276]:
df_duplicates = df[df['duplicates_to'] != -1].copy()
df_uniques = df[df['duplicates_to'] == -1].copy()

In [256]:
def custom_cosine_similarity(text_1: pd.Series, text_2: pd.Series):
    dot_product = np.dot(text_1, text_2)
    norm_text_1 = np.linalg.norm(text_1)
    norm_text_2 = np.linalg.norm(text_2)
    
    similarity = dot_product / (norm_text_1 * norm_text_2)
    
    return similarity

In [277]:
df_duplicates = pd.merge(left=df_duplicates,
                        right=df_uniques,
                        left_on='duplicates_to',
                        right_on='id',
                        suffixes=('__left', '__right'))
df_duplicates['label'] = 1
df_duplicates = df_duplicates.reset_index(drop=True)

df_duplicates.head()

,id__left,text__left,duplicates_to__left,text_embedded__left,id__right,text__right,duplicates_to__right,text_embedded__right,label
0,46407,enhancement current url querried another app ...,516502,"[-0.0023696667, -0.035115827, 0.062171057, -0....",516502,defect add applescript support getting curren...,-1,"[-0.00920859, 0.014499452, 0.032967843, 0.0078...",1
1,491005,defect powerpc applescript error get class cur...,516502,"[-0.006655017, 0.028761698, 0.03554143, 0.0072...",516502,defect add applescript support getting curren...,-1,"[-0.00920859, 0.014499452, 0.032967843, 0.0078...",1
2,95091,defect x86 helper application lack mozilla 0 9...,57420,"[-0.03253318, 0.011125067, 0.031311132, 0.0223...",57420,defect support helper app command line args u...,-1,"[-0.009637785, 0.00013978098, 0.023175225, 0.0...",1
3,308815,defect x86 linux choose helper application all...,57420,"[0.0041687232, 0.02763497, 0.012876986, 0.0068...",57420,defect support helper app command line args u...,-1,"[-0.009637785, 0.00013978098, 0.023175225, 0.0...",1
4,392205,enhancement add support arguments help apps u...,57420,"[-0.0012838916, 0.024868222, 0.021619998, 0.00...",57420,defect support helper app command line args u...,-1,"[-0.009637785, 0.00013978098, 0.023175225, 0.0...",1


In [278]:
half_rows = df_uniques.shape[0] // 2
df_uniques = df_uniques.sample(frac=1, replace=False, random_state=42)

df_uniques_temp = pd.DataFrame()
df_uniques_temp['text_embedded__left'] = df_uniques['text_embedded']
df_uniques_temp['text__left'] = df_uniques['text']
df_uniques_temp['id__left'] = df_uniques['id']

df_uniques_temp['text_embedded__right'] = df_uniques['text_embedded'].shift(1)
df_uniques_temp['text__right'] = df_uniques['text'].shift(1)
df_uniques_temp['id__right'] = df_uniques['id'].shift(1)

df_uniques_temp = df_uniques_temp.dropna()
df_uniques_temp['label'] = 0

df_uniques = df_uniques_temp.reset_index(drop=True)

del df_uniques_temp
gc.collect()

df_uniques.head()

,text_embedded__left,text__left,id__left,text_embedded__right,text__right,id__right,label
0,"[0.0064705536, 0.045026768, 0.014774143, 0.009...",defect x86 search without dialog new search di...,264962,"[0.015322351, 0.039216094, 0.018340139, 0.0095...",defect x86 getelementsbytagname confused short...,293692.0,0
1,"[0.0075292224, 0.022191392, 0.05013739, 0.0050...",defect unspecified colour updates needed new t...,1451474,"[0.0064705536, 0.045026768, 0.014774143, 0.009...",defect x86 search without dialog new search di...,264962.0,0
2,"[0.0017332855, 0.017958546, 0.028890675, 0.011...",defect x86 64 one word urlbar searches incredi...,972452,"[0.0075292224, 0.022191392, 0.05013739, 0.0050...",defect unspecified colour updates needed new t...,1451474.0,0
3,"[-0.0018321006, -0.034610473, 0.020652002, -0....",defect x86 firefox installer handle directory ...,341797,"[0.0017332855, 0.017958546, 0.028890675, 0.011...",defect x86 64 one word urlbar searches incredi...,972452.0,0
4,"[0.0012329025, 0.0037409784, 0.01816242, 0.001...",defect x86 firefox 18 0b1 b2 segmentation faul...,817482,"[-0.0018321006, -0.034610473, 0.020652002, -0....",defect x86 firefox installer handle directory ...,341797.0,0


In [284]:
df = pd.concat([df_duplicates, df_uniques], axis=0).sample(frac=1, replace=False, random_state=42).reset_index(drop=True)
gc.collect()

1596

### **5. Data Split**

### **5.1. Train Test Split**

In [286]:
X_train, X_test, y_train, y_test = train_test_split(df[['text_embedded__left', 'text_embedded__right']],
                                                    df['label'],
                                                    shuffle=True,
                                                    stratify=df['label'],
                                                    test_size=0.3,
                                                    random_state=42)

print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of y_test: {y_test.shape}')

Shape of X_train: (146384, 2)
Shape of X_test: (62737, 2)
Shape of y_train: (146384,)
Shape of y_test: (62737,)


### **6. Model Preparation**

In [287]:
input_1 = Input(shape=(300, ), name='input_1')
input_2 = Input(shape=(300, ), name='input_2')

shared_node_1 = Dense(256, name='shared_node_1')
shared_node_2 = Dense(256, name='shared_node_2')
shared_node_3 = Dense(128, name='shared_node_3')

x1 = shared_node_1(input_1)
x1 = shared_node_2(x1)
x1 = shared_node_3(x1)

x2 = shared_node_1(input_1)
x2 = shared_node_2(x2)
x2 = shared_node_3(x2)

cosine_similarity_layer = Dot(axes=-1, normalize=True, name='cosine_similarity')([x1, x2])
output_layer = Dense(1, activation='sigmoid', name='output')(cosine_similarity_layer)

training_model = Model(inputs=[input_1, input_2], outputs=output_layer)
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

training_model.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300)]                0         []                            
                                                                                                  
 shared_node_1 (Dense)       (None, 256)                  77056     ['input_1[0][0]',             
                                                                     'input_1[0][0]']             
                                                                                                  
 shared_node_2 (Dense)       (None, 256)                  65792     ['shared_node_1[0][0]',       
                                                                     'shared_node_1[1][0]']       
                                                                                           

In [288]:
inference_model = Model(inputs=input_1, outputs=cosine_similarity_layer)
inference_model.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300)]                0         []                            
                                                                                                  
 shared_node_1 (Dense)       (None, 256)                  77056     ['input_1[0][0]',             
                                                                     'input_1[0][0]']             
                                                                                                  
 shared_node_2 (Dense)       (None, 256)                  65792     ['shared_node_1[0][0]',       
                                                                     'shared_node_1[1][0]']       
                                                                                           

### **7. Training**

In [299]:
epochs = 10
batch_size = 32

training_model.fit(
    [np.vstack(X_train['text_embedded__left']), np.vstack(X_train['text_embedded__right'])],
    y_train,
    epochs=epochs,
    batch_size=batch_size
)

Epoch 1/10

4575/4575 [==============================] - 17s 3ms/step - loss: 0.5700 - precision_6: 0.1251 - recall_6: 0.2199
Epoch 2/10
4575/4575 [==============================] - 14s 3ms/step - loss: 0.3738 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 3/10
4575/4575 [==============================] - 15s 3ms/step - loss: 0.3738 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 4/10
4575/4575 [==============================] - 15s 3ms/step - loss: 0.3738 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 5/10
4575/4575 [==============================] - 16s 4ms/step - loss: 0.3738 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 6/10
4575/4575 [==============================] - 15s 3ms/step - loss: 0.3738 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 7/10
4575/4575 [==============================] - 15s 3ms/step - loss: 0.3738 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 8/10
4575/4575 [==============================] - 15s 3ms/step - loss:

In [301]:
val_predictions = training_model.predict([np.vstack(X_test['text_embedded__left']), np.vstack(X_test['text_embedded__right'])])
val_predictions_binary = (val_predictions > 0.5).astype(int)

precision = precision_score(y_test, val_predictions_binary)
recall = recall_score(y_test, val_predictions_binary)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')

1961/1961 [==============================] - 4s 2ms/step
Precision: 0.0000
Recall: 0.0000


d:\Programs\Miniconda3\envs\env_1\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
